In [2]:
import xarray as xr
from CASutils import filter_utils as filt 

In [3]:
# input: data array at specific lat and lon point
def remove_seasonality(data):
    data = data.sel(time=~((data.time.dt.month==2) & (data.time.dt.day == 29)))
    daystr = xr.DataArray(data.indexes['time'].strftime('%m-%d'), coords = data.time.coords, name="daystr")
    datseason = data.groupby(daystr).mean('time')
    harmonics = filt.calc_season_nharm(datseason, 4, dimtime=0)
    anoms = data.groupby(daystr)-harmonics
    return anoms.drop('daystr')

In [4]:
# input: lat,lon subset
def subtr_jja_means(data):
    jja_means = data.sel({'time':data['time.season']=='JJA'}).groupby('time.year').mean('time')
    return(data.groupby('time.year') - jja_means)

In [5]:
# input: lat,lon subset
def div_by_jja_std(data):
    sd_jja = data.sel({'time':data['time.season']=='JJA'}).std()
    return( data/sd_jja)    

In [6]:
# input: lat,lon subset
def standardized_by_jja_vals(data):
    
    means_removed = subtr_jja_means(data) # subtract JJA means
    means_rem_and_div_jja_std = div_by_jja_std(means_removed) # div JJA standard deviations
    
    return std_anoms

In [9]:
def linearly_detrend(data):
    beta = data.polyfit(dim='time', deg=1)
    fitted_slope = xr.polyval(data['time'], beta)
    data = data - fitted_slope['polyfit_coefficients']
    return(data)

In [8]:
def us_jja_anoms(data):
    data = data.sel({'lat': slice(25, 65), 'lon': slice(230, 295)})
    data = data.sel({'time': (data['time.month'] >= 6) & (data['time.month'] <= 8)})
    return(linearly_detrend(data))